In [28]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import requests
load_dotenv()


True

In [2]:
client = OpenAI(
    base_url = os.getenv('RUNPOD_CHATBOT_URL'),
    api_key = os.getenv('RUNPOD_TOKEN'),
)
model_name = os.getenv('MODEL_NAME')

In [7]:
client.base_url , client.api_key

(URL('https://api.runpod.ai/v2/glwuyzkizifp05/openai/v1/'),
 'rpa_B4Q1SXVLA3Q4XVDBIOU52G6BA04S5A45T0UU1Y7S1sazri')

In [4]:
response = client.chat.completions.create(
    model= "meta-llama/Meta-Llama-3-8B-Instruct",
    messages = [
        {"role": "user", "content": "what is the capital of germany?"}
    ],
    temperature = 0.0,
    top_p = 0.8,
    max_tokens = 2000,
)

In [5]:
response

ChatCompletion(id='chat-db6ac7319b6b42c1855414da025383ec', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The capital of Germany is Berlin.', role='assistant', function_call=None, tool_calls=[]), logprobs=None, stop_reason=None)], created=1736287685, model='meta-llama/Meta-Llama-3-8B-Instruct', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=8, prompt_tokens=17, total_tokens=25), prompt_logprobs=None)

In [6]:
response.choices[0].message.content

'The capital of Germany is Berlin.'

In [26]:
def get_chatbot_response(client, messages, temperature = 0.0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]}
        )

    response = client.chat.completions.create(
        model = "meta-llama/Meta-Llama-3-8B-Instruct",
        messages = input_messages,
        temperature = 0.0,
        top_p = 0.8,
        max_tokens = 2000,
    ).choices[0].message.content

    return response

### Prompt Engineering

1. Structured Output

In [10]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries. 

your iutput should be structured json format exactly like the one below. You are not allowed to write anything that the json object:
[
{
"Country:" The country that you will get capital of
"Capital" : The capital of country stated
}

]
"""


In [11]:
messages  = [{"role": "system", "content": system_prompt}]

In [12]:
messages.append({"role": "user", "content": "WHat's the capital of Germany?"})

In [13]:
messages

[{'role': 'system',
  'content': '\nYou are a helpful assistant that answer questions about capitals of countries. \n\nyour iutput should be structured json format exactly like the one below. You are not allowed to write anything that the json object:\n[\n{\n"Country:" The country that you will get capital of\n"Capital" : The capital of country stated\n}\n\n]\n'},
 {'role': 'user', 'content': "WHat's the capital of Germany?"}]

In [27]:
response = get_chatbot_response(client, messages,)
print(response)

[
{
"Country": "Germany",
"Capital": "Berlin"
}
]


In [30]:
json_response = json.loads(response)
json_response

[{'Country': 'Germany', 'Capital': 'Berlin'}]

2. Input Structuring

In [31]:
user_input = """
Get me the capitals of the following countries:
'''
1. Italy
2. Germany
3. France
'''
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": user_input})
response = get_chatbot_response(client , messages,)
print(response)

[
{
"Country": "Italy",
"Capital": "Rome"
},
{
"Country": "Germany",
"Capital": "Berlin"
},
{
"Country": "France",
"Capital": "Paris"
}
]


3. CoT (Chain-of-thought)

In [32]:
user_input = """
Calculate the result of this equation: 1+3

your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:

{
result: The final number resulted from calculating the equation above
}
"""

messages = [{"role": "user", "content": user_input}]
response = get_chatbot_response(client, messages)
print(response)


{
"result": 4
}


In [33]:
259/2*8654+91072*33

4126069.0

In [35]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,messages)

In [36]:
print(response)


{
    "steps": "First, calculate the division: 259/2 = 129.5. Then, multiply 129.5 by 8654: 129.5 * 8654 = 112,141.5. Next, multiply 91072 by 33: 91072 * 33 = 3,003,336. Finally, add the two results: 112,141.5 + 3,003,336 = 3,115,477.5.",
    "result": 3115477.5
}


In [37]:
4126069.0 - 3115477.5

1010591.5